This notebook establishes linkages between a WLCI literature database in Zotero and the xDD Digital Library (previously known as GeoDeepDive), using Digital Object Identifiers.  This process uses the Zotero API via a modified version of the pyzotero package to access information from the WLCI literature database.  The literature database is used in this case to help build a list of species that are referened by WLCI efforts (see Pull ITIS Names for xDD References.ipynb and Pull Species List from Zotero Items.ipynb).

The linkages made between the two databases are documented in Zotero by putting a link attachment back onto the Zotero library item. That establishes a base of operations for subsequent data extraction processes, keeping everything on established Zotero library items where continued curation can occur via web or desktop clients.

To operate this code, you will need to establish the ZOTERO_API_KEY environment variable in your Python environment. That key is tied to a given user account that has read/write permissions to the WLCI Library Group (established with the group ID parameter.

Note: This same idea should apply for any other Zotero library where we want to run the same overall process of linking a library reference to an xDD digital NLP representation. In the second part of this experiment, I will pick up from the established "xDD Document Link" identifier, pull back a list of associated species scientific names, and put those into a child attachment of the xDD Document Link attachment. This then begins to lay out structured annotation retrieved via algorithms directly into the Zotero library items, adding value to those items and using the Zotero library as a living cloud-based repository with both human and software generated value.

In [1]:
from pyzotero import zotero
import os
import requests
from IPython.display import display

In [2]:
wlci_library_group_id = "2341914"

In [3]:
wlci_lib = zotero.Zotero(wlci_library_group_id, "group", os.environ["ZOTERO_API_KEY"])

In [4]:
wlci_lib_items = wlci_lib.everything(wlci_lib.top())

In [5]:
print(len(wlci_lib_items))

300


In [6]:
set([i["data"]["itemType"] for i in wlci_lib_items])

{'attachment',
 'book',
 'document',
 'journalArticle',
 'presentation',
 'report',
 'thesis',
 'videoRecording',
 'webpage'}

In our current work with the xDD library, we are only working on those articles for which we were able to establish a DOI in our Zotero library. DOIs either came from the original article metadata imported into the library and are in the DOI property, or they were added as a note and will show up in the extra property for now. (Need to revisit how we manage this in the library.) To work these over, I assemble a list of Zotero IDs and DOIs.

In [12]:
lookup_doi_list = [(i["data"]["key"],i["data"]["DOI"]) for i in wlci_lib_items if "DOI" in i["data"].keys() and len(i["data"]["DOI"]) > 0]
lookup_doi_list.extend([(i["data"]["key"],i["data"]["extra"]) for i in wlci_lib_items if "DOI" not in i["data"].keys() and 'extra' in i['data'].keys() and len(i["data"]["extra"]) > 0 and i["data"]["extra"].split(":")[0] != "OCLC"] )

We will eventually pull these functions out into our Python package for this work. Here we consult xdd using xdd_api and we use a helper function to assemble the necessary information into the Zotero template for the attachment.

We had to fork the pyzotero package and create a [branch](https://github.com/skybristol/pyzotero/tree/attachment-template-type) with an adjustment to the item_template function, which was failing on the "attachment" template type and needed an additional parameter against the Zotero REST API. This was posted back to the project as a pull request for consideration by Sky.

In [13]:
def xdd_api(route, params):
    """Create list of docs mentioning a term of interest
    Parameters : see https://geodeepdive.org/api for more detail
    ----------
    routes : str of available api routes for xDD 
    params : str of key value pairs of paramaters:values separated by &
    """
    base_url = 'https://geodeepdive.org/api'
    search = (base_url + '/' + route + '?' + str(params))
    r=requests.get(search)
    if r.status_code == 200 and 'success' in r.json():
        json_r = r.json()
        data = json_r['success']['data']
        return data
    elif r.status_code == 200:
        return None
    else:
        return None

    
def xdd_link_attachment(parent_id, xdd_record, template):
    template["parentItem"] = parent_id
    template["title"] = "xDD Document Link"
    template["url"] = f"https://geodeepdive.org/api/articles?docid={xdd_record['_gddid']}"
    template["accessDate"] = "CURRENT_TIMESTAMP"
    template["note"] = "Link to xDD document established through search algorithm"
    template["tags"] = ["xdd_doc_link"]
    return template

Here I run through the list of DOIs and Zotero IDs to check xDD for a link. This could be rearranged in a variety of ways and will need to be explored further for production use. I do check to make sure we don't already have a link of the appropriate "type" (based on assigning a particular title to the attachment).

In [14]:
for ref in lookup_doi_list:
    print(ref[0], ref[1])
    xdd_data = xdd_api(
        'articles', 
        'max=1&doi='+str(ref[1])
    )

    if xdd_data is not None:
        child_items = wlci_lib.children(ref[0])
        current_xdd_doc_attachment = next((i for i in child_items if i["data"]["title"] == "xDD Document Link"), None)
        
        if current_xdd_doc_attachment is None:
            xdd_doc_attachment = xdd_link_attachment(ref[0], xdd_data[0], wlci_lib.item_template("attachment", linkmode="linked_url"))
            create_response = wlci_lib.create_items([xdd_doc_attachment])
            if not create_response["successful"]:
                display(create_response)
        else:
            display(current_xdd_doc_attachment)


NNI3PJ5Z 10.1111/1365-2664.12513


{'key': '2QVF3BJH',
 'version': 3159,
 'library': {'type': 'group',
  'id': 2341914,
  'name': 'WLCI',
  'links': {'alternate': {'href': 'https://www.zotero.org/groups/wlci',
    'type': 'text/html'}}},
 'links': {'self': {'href': 'https://api.zotero.org/groups/2341914/items/2QVF3BJH',
   'type': 'application/json'},
  'alternate': {'href': 'https://www.zotero.org/groups/wlci/items/2QVF3BJH',
   'type': 'text/html'},
  'up': {'href': 'https://api.zotero.org/groups/2341914/items/NNI3PJ5Z',
   'type': 'application/json'}},
 'meta': {'createdByUser': {'id': 1119084,
   'username': 'skybristol',
   'name': 'Sky Bristol',
   'links': {'alternate': {'href': 'https://www.zotero.org/skybristol',
     'type': 'text/html'}}}},
 'data': {'key': '2QVF3BJH',
  'version': 3159,
  'parentItem': 'NNI3PJ5Z',
  'itemType': 'attachment',
  'linkMode': 'linked_url',
  'title': 'xDD Document Link',
  'accessDate': '2019-09-28T17:42:09Z',
  'url': 'https://geodeepdive.org/api/articles?docid=5897848bcf58f1ac

88PSMFIC 10.1016/j.biocon.2015.02.009


{'key': 'JV8EH3GN',
 'version': 3160,
 'library': {'type': 'group',
  'id': 2341914,
  'name': 'WLCI',
  'links': {'alternate': {'href': 'https://www.zotero.org/groups/wlci',
    'type': 'text/html'}}},
 'links': {'self': {'href': 'https://api.zotero.org/groups/2341914/items/JV8EH3GN',
   'type': 'application/json'},
  'alternate': {'href': 'https://www.zotero.org/groups/wlci/items/JV8EH3GN',
   'type': 'text/html'},
  'up': {'href': 'https://api.zotero.org/groups/2341914/items/88PSMFIC',
   'type': 'application/json'}},
 'meta': {'createdByUser': {'id': 1119084,
   'username': 'skybristol',
   'name': 'Sky Bristol',
   'links': {'alternate': {'href': 'https://www.zotero.org/skybristol',
     'type': 'text/html'}}}},
 'data': {'key': 'JV8EH3GN',
  'version': 3160,
  'parentItem': '88PSMFIC',
  'itemType': 'attachment',
  'linkMode': 'linked_url',
  'title': 'xDD Document Link',
  'accessDate': '2019-09-28T17:44:07Z',
  'url': 'https://geodeepdive.org/api/articles?docid=57a84443cf58f170

G9L9FU5W 10.1016/j.ecolmodel.2017.05.017


...

{'key': 'IKSTJD4K',
 'version': 3200,
 'library': {'type': 'group',
  'id': 2341914,
  'name': 'WLCI',
  'links': {'alternate': {'href': 'https://www.zotero.org/groups/wlci',
    'type': 'text/html'}}},
 'links': {'self': {'href': 'https://api.zotero.org/groups/2341914/items/IKSTJD4K',
   'type': 'application/json'},
  'alternate': {'href': 'https://www.zotero.org/groups/wlci/items/IKSTJD4K',
   'type': 'text/html'},
  'up': {'href': 'https://api.zotero.org/groups/2341914/items/WJESVJUN',
   'type': 'application/json'}},
 'meta': {'createdByUser': {'id': 1119084,
   'username': 'skybristol',
   'name': 'Sky Bristol',
   'links': {'alternate': {'href': 'https://www.zotero.org/skybristol',
     'type': 'text/html'}}}},
 'data': {'key': 'IKSTJD4K',
  'version': 3200,
  'parentItem': 'WJESVJUN',
  'itemType': 'attachment',
  'linkMode': 'linked_url',
  'title': 'xDD Document Link',
  'accessDate': '2019-09-28T17:59:05Z',
  'url': 'https://geodeepdive.org/api/articles?docid=558edc9ae1382310

DSJ4Z9E7 10.3133/ds524
5RYDFKE5 10.3133/fs20173030
RJ6VU7QH 10.3133/ofr20181188


{'key': 'DVPZ36KR',
 'version': 3202,
 'library': {'type': 'group',
  'id': 2341914,
  'name': 'WLCI',
  'links': {'alternate': {'href': 'https://www.zotero.org/groups/wlci',
    'type': 'text/html'}}},
 'links': {'self': {'href': 'https://api.zotero.org/groups/2341914/items/DVPZ36KR',
   'type': 'application/json'},
  'alternate': {'href': 'https://www.zotero.org/groups/wlci/items/DVPZ36KR',
   'type': 'text/html'},
  'up': {'href': 'https://api.zotero.org/groups/2341914/items/RJ6VU7QH',
   'type': 'application/json'}},
 'meta': {'createdByUser': {'id': 1119084,
   'username': 'skybristol',
   'name': 'Sky Bristol',
   'links': {'alternate': {'href': 'https://www.zotero.org/skybristol',
     'type': 'text/html'}}}},
 'data': {'key': 'DVPZ36KR',
  'version': 3202,
  'parentItem': 'RJ6VU7QH',
  'itemType': 'attachment',
  'linkMode': 'linked_url',
  'title': 'xDD Document Link',
  'accessDate': '2019-09-28T17:59:07Z',
  'url': 'https://geodeepdive.org/api/articles?docid=5d23b0bf0b45c76c

IFRB23U7 10.3133/ofr20091274
UKX4YSWG 10.3133/ds510
CEPBNS8E 10.3133/sir20105067
WVAXM5H4 10.3133/ds891
Z7LFWN5E 10.3133/ofr20161147


{'key': 'Z25JSPTZ',
 'version': 3203,
 'library': {'type': 'group',
  'id': 2341914,
  'name': 'WLCI',
  'links': {'alternate': {'href': 'https://www.zotero.org/groups/wlci',
    'type': 'text/html'}}},
 'links': {'self': {'href': 'https://api.zotero.org/groups/2341914/items/Z25JSPTZ',
   'type': 'application/json'},
  'alternate': {'href': 'https://www.zotero.org/groups/wlci/items/Z25JSPTZ',
   'type': 'text/html'},
  'up': {'href': 'https://api.zotero.org/groups/2341914/items/Z7LFWN5E',
   'type': 'application/json'}},
 'meta': {'createdByUser': {'id': 1119084,
   'username': 'skybristol',
   'name': 'Sky Bristol',
   'links': {'alternate': {'href': 'https://www.zotero.org/skybristol',
     'type': 'text/html'}}}},
 'data': {'key': 'Z25JSPTZ',
  'version': 3203,
  'parentItem': 'Z7LFWN5E',
  'itemType': 'attachment',
  'linkMode': 'linked_url',
  'title': 'xDD Document Link',
  'accessDate': '2019-09-28T17:59:09Z',
  'url': 'https://geodeepdive.org/api/articles?docid=57d99162cf58f191